Pierre Navaro - [Institut de Recherche Mathématique de Rennes](https://irmar.univ-rennes1.fr) - [CNRS](http://www.cnrs.fr/)

* [Interactive Parallel Computing in Python](https://github.com/ipython/ipyparallel)
* [Documentation](https://ipyparallel.readthedocs.io/en/latest/)


Install ipyparallel:

    pip3 install ipyparallel

To enable the `IPython Clusters` tab in Jupyter Notebook:

    ipcluster nbextension enable


To disable it again:

    ipcluster nbextension disable

See the [documentation on configuring the notebook server](https://jupyter-notebook.readthedocs.io/en/latest/public_server.html)
to find your config or setup your initial `jupyter_notebook_config.py`.


## Run

Start a cluster:

    ipcluster start

Use it from Python:

```python
import os
import ipyparallel as ipp

rc = ipp.Client()
ar = rc[:].apply_async(os.getpid)
pid_map = ar.get_dict()
```

In [12]:
%%file psum.py
from mpi4py import MPI
import numpy as np

def psum(a):
    locsum = np.sum(a)
    rcvBuf = np.array(0.0,'d')
    MPI.COMM_WORLD.Allreduce([locsum, MPI.DOUBLE],
        [rcvBuf, MPI.DOUBLE],
        op=MPI.SUM)
    return rcvBuf

Writing psum.py


In [15]:
from IPython.parallel import Client
c = Client(profile='mpi')
view = c[:]
view.activate()
view.run('psum.py')

<AsyncResult: execute>

In [17]:
import numpy as np
view.scatter('a',np.arange(16,dtype='float'))

<AsyncResult: scatter>

In [11]:
import mpi4py.MPI as mpi

rank = mpi.COMM_WORLD.rank
size = mpi.COMM_WORLD.size 

if rank == 0:
	valeur = 1000
	mpi.COMM_WORLD.send(valeur, dest = 1) 
	valeur = mpi.COMM_WORLD.recv(source = size-1) 
else:
	valeur = mpi.COMM_WORLD.recv(source = rank-1) 
	valeur += 1
	mpi.COMM_WORLD.send(valeur, dest = (rank+1)%size) 

print ("Moi, Processus,", rank," j'ai recu", valeur, "du processus ")


Exception: MPI_ERR_RANK: invalid rank